# 大模型API

- 云端大模型：https://bailian.console.aliyun.com/
- 本地大模型：https://qwen.readthedocs.io/zh-cn/latest/getting_started/quickstart.html

In [1]:
import pandas as pd
import glob, json, re
import numpy as np
from modelscope import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"  #

In [2]:
model_name = '/DATA/disk0/public_model_weights/qwen/Qwen2.5-0.5B-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def basechat(messages:list) -> str:
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def get_chunk_list(lst, chunk_size=200):
    """Yield successive chunk_size chunks from lst."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


def exteract_module_raw_text(module_name, lines):
    messages=[
        {"role": "system", "content": f"""你是一个专业的保险行业的信息处理专家，请对下面的文本中抽取出与{module_name}相关的文本，只需要输出原文，不要有其他输出。如果相关的本文本不存在，则输出一个空格。"""},
        {"role": "user", "content": "".join(lines)},
    ]
    return basechat(messages)

def analysis_conflict(module_name, content1, content2):
    messages=[
        {"role": "system", "content": f"""你是一个专业的保险行业的信息处理专家，请对下面文本进行严谨的一致性进行分析，判断是否相同条件的表达存在不一致的情况，只需要回答一致或不一致，不要有其他任何输出。
            
- 基础产品销售信息：该保险产品的基础配置信息，包括产品名、附加的条款信息、销售限制等；
- 投保条款：投保过程中的缴费约定、投被保人条件限制等；
- 保障责任：约定该产品的保险责任细节，如保障范围、保险金额、增值服务等；
- 保障相关时间：约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等；
- 赔付 & 领取规则：约定该产品的保险责任的赔付、给付、领取及免赔细节，如赔付年龄/比例/次数等；
- 责任免除：约定该产品不承担保险责任的情形；
- 续保条款：约定续保相关信息，包括但不限于续保条件、保证续保等；
- 退保条款：约定退保相关信息，包括但不限于退保条件、退保手续费等；
- 出险条款：约定出险相关信息，包括但不限于出险地点、出险方式等；
- 附加条款：约定该产品的附加条款，如特别约定等；
- 术语解释：约定该产品的术语解释，如名词定义等；"""},
        {"role": "user", "content": f"""片段1\n{content1}\n\n片段2\n{content2}\n\n分析上述两个片段的{module_name}是否存在相同的条件但表达不一致的情况。"""},
    ]
    return basechat(messages)


In [4]:
for row in pd.read_json("../DATA/样例集/data.jsonl", lines=True).iloc[2:3].iterrows():
    module_name = row[1].rule.replace("该产品的", "").replace("在各材料中的定义没有冲突", "")

    module_content_list = []
    for path in glob.glob(f"../DATA/样例集/materials/{row[1].material_id}/*/*"):
        lines = open(path).readlines()

        module_lines = ""
        for chunk_lines in get_chunk_list(lines):
            res = exteract_module_raw_text(module_name, chunk_lines)
            if res not in ['', '空格']:
                module_lines += res
        if module_lines:
            module_content_list.append(module_lines)

    print("\n")
    print(row[1].material_id)
    print(module_content_list)
    for i in range(len(module_content_list)):
        if i == len(module_content_list) - 1: break
        for j in range(i+1, len(module_content_list)):
            print(analysis_conflict(module_name, module_content_list[i], module_content_list[j]))



m_00003s
['投保须知责任免除山东省 (0530)3328580', '一、责任免除事项', '责任免除', '1/11 \n恒安标准人寿[2023]疾病保险 001 号  \n请扫描以查询验证条款  \n恒 安 标 准 人 寿 保 险 有 限 公 司  \nHeng An Standard Life Insurance Company Limited  \n恒安标准爱佑保乐享版恶性肿瘤疾病保险（互联网）条款  \n目  录  \n1. 您与我们订立合同时需要了解的内容  \n1.1. 合同的种类及构成  \n1.2. 投保条件  \n1.3. 保险期间  \n1.4. 保险责任的开始和终止  \n1.5. 犬然期  \n1.6. 明确说明与如实告知  \n2. 合同项下的保障和利益  \n2.1. 基本保险金额  \n2.2. 等待期  \n2.3. 我们提供的保障  \n2.4. 我们不承担的责任  \n3. 您在合同项下的权利和义务  \n3.1. 保险费的交付  \n3.2. 宽限期  \n3.3. 合同效力的中止和恢复  \n3.4. 犬然期后解除合同的处理  \n3.5. 合同内容的变更  \n3.6. 现金价值  \n4. 如如何申请领取保险金  \n5/11 \n4.1. 保险金受益人  \n5/11 \n1. 您与我们订立合同时需要了解的内容  \n1.1. 合同的种类及构成  \n1.2. 投保条件  \n1.3. 保险期间  \n1.4. 保险责任的开始和终止  \n1.5. 犮然期  \n1.6. 明确说明与如实告知  \n2. 合同项下的保障和利益  \n2.1. 基本保险金额  \n2.2. 等待期  \n2.3. 我们提供的保障  \n2.4. 我们不承担的责任  \n3. 您在合同项下的权利和义务  \n3.1. 保险费的交付  \n3.2. 宽限期  \n3.3. 合同效力的中止和恢复  \n3.4. 犮然期后解除合同身故保险金作为被保险人的遗产处理时，被保险人的继承人除应提供上述证明材料外，还应向我们\n提供证明其继承权的有效公证文书或法律文件。淋巴结转移']
一致
一致
不一致
一致
不一致
一致


In [ ]:
for row in pd.read_json("../DATA/测试 A 集/data.jsonl", lines=True).iloc[:].iterrows():
    module_name = row[1].rule.replace("该产品的", "").replace("在各材料中的定义没有冲突", "")

    print("")
    print(row[1].material_id)

    try:
        module_content_list = []
        for path in glob.glob(f"../DATA/测试 A 集/materials/{row[1].material_id}/*/*"):
            print(path)
            lines = open(path).readlines()
            module_lines = ""
            for chunk_lines in get_chunk_list(lines):
                res = exteract_module_raw_text(module_name, chunk_lines)
                if res:
                    module_lines = module_lines + res
            
            module_content_list.append(module_lines)
    
        result = [0, 1]
        for i in range(len(module_content_list)):
            for j in range(i, len(module_content_list)):
                if "不一致" in analysis_conflict(module_name, module_content_list[i], module_content_list[j]):
                    result.append(1)
                else:
                    result.append(0)
    
        with open("submit.jsonl", "a") as up:
            up.write(json.dumps({
                "material_id": row[1].material_id,
                "rule_id": row[1].rule_id,
                "result": bool(np.mean(result) < 0.3)
            }) + "\n")
    
        print(np.mean(result))
    except:
        with open("submit.jsonl", "a") as up:
            up.write(json.dumps({
                "material_id": row[1].material_id,
                "rule_id": row[1].rule_id,
                "result": False
            }) + "\n")


m_00001a
../DATA/测试 A 集/materials/m_00001a/INSURE_NOTICE/0.md
../DATA/测试 A 集/materials/m_00001a/HEAD_IMG/0.md
../DATA/测试 A 集/materials/m_00001a/INTRODUCE_IMG/0.md
../DATA/测试 A 集/materials/m_00001a/LIABILITY_EXCLUSION/0.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/3.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/7.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/1.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/2.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/0.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/8.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/6.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/4.md
../DATA/测试 A 集/materials/m_00001a/CLAUSE/5.md
0.34408602150537637

m_00002a
../DATA/测试 A 集/materials/m_00002a/INSURE_NOTICE/0.md


In [2]:
from openai import AsyncOpenAI, OpenAI


In [5]:
class DeepChat:

    def __init__(self):
        DEEPSEEK_API_KEY = 'sk-c6cd584debfa4eba866e430fab25252a'
        self.client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

    def chat(self, prompt:str='你好'):
        messages = [{"role": "user", "content": prompt}]
        response = self.client.chat.completions.create(
                model="deepseek-chat", messages=messages
        ).choices[0].message.content
        return response

In [6]:
deepai = DeepChat()

In [7]:
# 免责声明
prompt_ = "提取出下面文本的“责任免除”（约定该产品不承担保险责任的情形），要完整的、不要有遗漏的信息，且不要修改数据："
info = ""






'你好！😊 很高兴见到你～有什么我可以帮你的吗？'

In [17]:
prompt = """提取出下面文本的“免责声明”，要完整的、不要有遗漏的信息，且不要修改数据：
投保须知及声明
一、投保须知
1  .  投保地  区
本保险产品由泰康在线财产保险股份有限公司承保，在中华人民共和国境内（港、澳、台地区除外）开展与互联网交易直接相关的保险业务，不设分支机构，各项保险服务流程均可线上操作。本计划仅限在中国大陆有固定居住地（常住）的人士投保。
2.保单形式
网上投保为您提供电子保单，根据《中华人民共和国民法典》规定，数据电文是合法的合同表现形式，电子保单与纸质保单具有同等法律效力。您可以登录www.tk.cn自助查询对电子保单的真实性进行验证。
3.如实告知
您应如实填写投保信息，并就我们提出的询问据实告知，否则我们有权根据《中华人民共和国保险法》第十六条的规定解除保险合同且不承担赔偿责任：
（1）订立保险合同时，保险公司就保险标的或者被保险人的有关情况提出询问的，投保人应当如实告知。
（2）投保人故意或者因重大过失未履行前款规定的如实告知义务，足以影响保险公司决定是否同意承保或者提高保险费率的，保险公司有权解除合同。
（3）投保人故意不履行如实告知义务的，保险公司对于合同解除前发生的保险事故，不承担赔偿责任，并不退还保险费。
（4）投保人因重大过失未履行如实告知义务，对保险事故的发生有严重影响的，保险公司对于合同解除前发生的保险事故，不承担赔偿责任，但退还保险费。
4.相关授权
  如您投保此保险，视为您本人授权泰康在线出于提升保险服务质量之目的，合法的从任何单位、组织和个人获取本人有关的资料和证明，作为审核投保、处理理赔及提供与本保险有关其他服务的依据。
5.信息变更
如果您的邮件地址、通信地址、邮编、联系电话发生变化，请与本公司客户服务电话95522-3联系，办理变更事宜。
6  .偿付能力及风险综合评价
我公司最新季度综合偿付能力充足率、风险综合评级评价结果等信息，请登录我公司偿付能力信息披露页面
7  .理赔入口
（1）理赔报案：支付宝搜索“泰在线保险服务小程序”；（2）提供资料：使用在线理赔服务，提交理赔申请资料；（3）审核资料：收到理赔资料后，理赔人员将会对资料进行审核和调查；（4）完成理赔：根据审核结果，将赔款直接支付至被保人或受益人的指定账户。

二、产品说明
1.   本产品由 泰康在线财产保险股份有限公司 承保，适用条款为《泰康在线财产保险股份有限公司少儿医疗费用保险（互联网专属） 》 ， 注册号 ： C00019932512023021426563 ；《泰康在线财产保险股份有限公司 个人 白血病疾病保险（ 互联网专属） 》，注册号： C00019932612021120910553 ；《泰康在线财产保险股份有限公司人身意外伤害保险 E款 （ 互联网专属 ）》，注册号： C00019932312021122840523 ；《 泰康在线财产保险股份有限公司附加意外伤害救护车费用保险（互联网专属）》，注册号：C00019932522021122225183；《泰康在线财产保险股份有限公司骨折意外伤害保险（互联网专属）条款》，注册号：C00019932312021122225153；《泰康在线财产保险股份有限公司附加未成年人责任保险条款》，注册号：C00019930922020041301962。
2.   本产品的交费方式为 月交 或者 年交 :
（ 1 ） 选择 年交 保险费的，应当在合同成立时一次性交清保险费。保险费未交清前，本合同不生效，对本合同生效前发生的保险事故，保险公司不承担保险责任；
（ 2 ） 选择 月交 保险费的， 投保 人应当在合同成立时交纳首期保险费，若未按约定交纳首期保险费，本合同不生效，对本合同生效前发生的保险事故，保险人不承担保险责任。
投保 人在交纳首期保险费后，应当在每个约定的交费日交纳其余各期保险费 ， 约定交费日为保单生效日在各自然月的对应日期，若当月无对应日期，则为该自然月最后一日 。
若投保人未按约定交纳其余各期保险费，保险人允许投保人在约定交费日起三十日（含第三十日）内补交保险费，如果被保险人在此三十日内发生保险事故，保险人将扣减投保人欠交的保险费后按主险合同约定承担赔偿保险金的责任 。 如果 投保 人在约定交费日起三十日（含第三十日）内未补交保险费，本合同自上述期限届满之日二十四时起效力中止，自本合同效力中止日起发生的保险事故或医疗费用，保险公司不承担保险责任；
因投保人未按约定交纳保险费导致 本 合同效力 中 止，投保人要求保单复效的，投保人可重新申请投保新单，经保险人审核同意后，新保险合同生效，不重新计算等待期 。
3.   本产品被保险人年龄限制为出生满30天（含）-17周岁（含），投保人必须为其父母；
4.   社会基本医疗保险：是指包括新型农村合作医疗保险、城镇职工基本医疗保险、城镇居民基本医疗保险等政府举办的基本医疗保障项目；
5.   同一保险期限内，本产品最多可购买一份，多投超出部分保单自始无效；
6.   本产品自保单生效起15天为犹豫期，投保人在此期间提出解除本合同，保险人将无息退还投保人所支付的全部保险费；犹豫期后申请解除本合同会遭受一定经济损失；
7.   本产品网上投保申请日后次日零时生效，门（急）诊医疗费用保险金等待期为30天（含犹豫期），住院医疗保险金等待期为30天（含犹豫期），个人白血病保险金等待期90天，续保无等待期；
8.   本产品门（急）诊医疗费用保险金相关约定：
（1）门（急）诊医疗费用保险金每次事故绝对免赔额为200元；每次事故绝对免赔额指的是单次门（急）诊医疗费用绝对免赔额。单次门诊医疗指被保险人同一天在同一医院同一科室所进行的门急诊治疗；
（2）门（急）诊医疗费用保险金保险金额为5000元，每次事故赔偿限额为500元；
（3）在本合同保险期间内，被保险人遭受意外伤害事故或在等待期后经医院确诊罹患疾病，并经二级及以上公立医院（不含国际医疗及特需部）诊断必须进行必要的门（急）诊治疗的，其门（急）诊（不包括急诊留院观察期）医疗费用是属于符合当地社会基本医疗保险支付范围内的医疗费用，当发生的单次门急诊医疗费用金额超过100元时，保险人承担如下保险责任：
① 若被保险人已从其他途径（包括但不限于社会基本医疗保险、公费医疗、工作单位、侵权人或侵权责任承担方、保险人在内的任何商业保险机构等）获得本合同责任范围内的医疗费用补偿的，赔偿比例为100%。保险人按以下公式进行赔偿：
门（急）诊医疗费用保险金=（每次发生的社保支付范围内的医疗费用—每次已从其他途径获得的医疗费用补偿—单次免赔额）×100%；
② 若被保险人未从其他途径（包括但不限于社会基本医疗保险、公费医疗、工作单位、侵权人或侵权责任承担方、保险人在内的任何商业保险机构等）获得本合同责任范围内的医疗费用补偿的，赔偿比例为60%。保险人按以下公式进行赔偿：
门（急）诊医疗费用保险金=（每次发生的社保支付范围内的医疗费用—单次免赔额）×60%
③ 被保险人的社保卡个人账户部分支出视为个人支付，不属于已获得的医疗费用补偿，不进行前述扣除。
（4）门（急）诊医疗费用保险金的累计赔偿次数以10次为限，累计赔偿次数超过10次，不予赔付。
9.   本产品住院医疗保险金相关约定：
（1）住院医疗保险金保险金额为1万元，无免赔额；
（2）在本合同保险期间内，被保险人因意外伤害事故或在等待期后罹患疾病，在医院经专科医生诊断必须住院治疗的，保险人对被保险人在住院期间实际支出的属于当地社会基本医疗保险支付范围内的必需且合理的住院医疗费用，保险人承担如下保险责任：被保险人以参加社会基本医疗保险或公费医疗身份就诊并结算的，赔偿比例为100%；被保险人未以参加社会基本医疗保险或公费医疗身份就诊并结算的，赔偿比例为60%；
本产品附加未成年人责任保险金，财产损失每次事故的绝对免赔额为实际损失的10%；人身伤害每次事故的绝对免赔额为100；
10.   本产品附加意外伤害救护车费用保险金，每次事故的免赔额为0，赔付比例100%；
11.   本产品骨折意外伤害保险金，保险人根据被保险人的骨折程度及本合同附录《人身保险骨折程度与保险金给付比例表》确定给付比例，并按所确定的给付比例乘以保险单上载明的骨折意外伤害保险金额向被保险人给付骨折意外伤害保险金。
12.   本产品医疗相关保障限定为中华人民共和国境内合法经营的二级以上（含二级）公立医院普通部（不含国际医疗及特需部)，以及保险人扩展的承保医院。同时保险人不予承保被保险人在河北省承德市兴隆县中医院、北京市平谷区、密云县和怀柔区的所有医疗机构、河南省通许县的所有医疗机构所发生的医疗保险责任；
13.   本产品为不保证续保合同。本产品保险期间为1年。保险期间届满或保险期间届满前30日内，投保人需要重新向保险人申请投保本产品，并经保险人同意，交纳保险费，获得新的保险合同；
14.   投保年龄（投保时被保险人年龄）是健康险产品定价的重要影响因素，故年龄不同对应的投保保费或者续保保费也有可能存在一定差异 。

三  、  免责  声明
《  泰康  在线 财产保险股份有限公司少儿医疗  费用  保险（  互联网专属  ）  》  免责说明
因下列情形之一导致被保险人支出医疗费用的，保险人不承担赔偿保险金的责任：
（一）  投保人对被保险人的故意杀害、故意伤害；
（二）  投保人或被保险人在投保时未如实告知的疾病；被保险人在投保前罹患且在投保时尚未治愈的疾病；
（三）  被保险人故意自伤，但被保险人自伤时为无民事行为能力人的除外；
（四）  被保险人故意犯罪或者抗拒依法采取的刑事强制措施，或被国家机关依法拘禁或服刑期间伤病；
（五）  被保险人感染艾滋病病毒或者患艾滋病、性病、精神和行为障碍、遗传性疾病、先天性畸形、变形或者染色体异常（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类》 第十次修订版（ICD-10） 为准）；
（六）  《中华人民共和国传染病防治法》规定的甲类及乙类法定传染病（不含病毒性肝炎），或者国家有关法律、法规、规范性法律文件规定的法定传染病。前述传染病定义以《中华人民共和国传染病防治法》或者国家有关法律、法规、规范性法律文件的规定为准；
（七）  被保险人接受疗养、康复治疗、心理治疗、美容、视力矫正手术、各种健康体检项目及预防性医疗项目、各种矫形及生理缺陷的手术和检查治疗项目、各种健美治疗项目、牙科保健及牙齿治疗以及非意外伤害事故所致的整容、整形手术；
（八）  被保险人接受如下项目的治疗：皮肤色素沉着；雀斑、痣的治疗和去除；对浅表静脉曲张、蜘蛛脉、除瘢痕疙瘩型外的其他瘢痕、纹身去除、皮肤变色的治疗或手术；包皮环切术、包皮剥离术、包皮气囊扩张术、性功能障碍治疗引起的医疗费；
（九）  被保险人患有性早熟、佝偻病、身材矮小、骨发育异常及其他发育迟缓疾患；
（十）  被保险人接受各类医疗鉴定，包括但不限于医疗事故鉴定、精神病鉴定、验伤鉴定、亲子鉴定、遗传基因鉴定所发生的费用；
（十一）  被保险人接受实验性治疗，即未经科学或医学认可的医疗；体外或植入的医疗辅助装置或用具（义肢、轮椅、拐杖、听器、眼镜或隐形眼镜、义眼等）及其安装费用；
（十二）  被保险人殴斗、醉酒，主动吸食或注射毒品，违反规定使用麻醉或精神药品；
（十三）  被保险人挑衅或者故意行为而导致的打斗、被袭击、被谋杀；
（十四）  被保险人酒后驾驶、无合法有效驾驶证驾驶，或者驾驶无合法有效行驶证的机动车导致交通意外引起的医疗费用；
（十五）  被保险人从事高风险运动导致的伤害引起的治疗；
（十六）  被保险人未遵医嘱私自服用、涂用、注射药物；
（十七）  被保险人由于职业病、医疗事故引起的医疗费用；
（十八）  战争、军事行动、暴乱、武装叛乱、核爆炸、核辐射、核污染；
（十九）  未经医生处方自行购药或非医院药房购药，医生开具的单次处方超过30天部分的药品费用。
《  泰康  在线 财产保险股份有限公司  个人白血病疾病  保险（  互联网专属  ）  》  免责说明
因下列情形之一导致被保险人罹患本合同定义的白血病的，保险人不承担给付保险金的责任：
（一）  投保人对被保险人的故意杀害、故意伤害；
（二）  被保险人故意自伤、故意犯罪或者抗拒依法采取的刑事强制措施或被国家机关依法拘禁或服刑期间伤病；
（三）  被保险人酗酒、殴斗、主动吸食或注射毒品，违反规定使用麻醉或精神药品；
（四）  战争（无论宣战与否）、军事冲突、暴乱或武装叛乱；
（五）  核爆炸、核辐射或核污染；
（六）  被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）、高血压III级期间；
（七）  被保险人患遗传性疾病，先天性畸形、变形或染色体异常（依据世界卫生组织《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）确定）；
（八）  未经科学或者医学认可的预防性治疗、实验性或试验性治疗及其产生的后果所导致的恶性肿瘤；
（九）  被保险人在投保时已经患有白血病的，但投保时保险人已知晓并做出书面认可的除外；
（十）  由于职业病、医疗事故引起 。
《  泰康在线财产保险股份有限公司人身意外伤害保险  E  款  （  互联网专属）  ）》  免责说明：  因下列原因造成被保险人身故、伤残的，保险人不承担给付保险金责任：
（一）  投保人的故意行为；
（二）  被保险人自伤或自杀，但被保险人自杀时为无民事行为能力人的除外；
（三）  因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；
（四）  被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；
（五）  被保险人未遵医嘱，私自服用、涂用、注射药物；
（六）  核爆炸、核辐射或核污染等放射性污染；
（七）  恐怖袭击；
（八）  被保险人犯罪或抗拒依法采取的刑事强制措施；
（九）  被保险人从事跳伞、滑翔、赛马、马术表演、赛车、拳击等高风险运动或活动；
（十）  被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；
（十一）  被保险人因精神错乱或失常而导致的意外；
（十二）  被保险人非法搭乘交通工具或搭乘未经保险事故发生地相关政府部门登记许可的交通工具；
（十三）  被保险人因接受检查、麻醉、整容手术及其它内外科手术、药物治疗等导致的意外；
（十四）  被保险人因意外伤害事故以外的原因失踪而被法院宣告死亡的。
被保险人在下列期间遭受意外伤害导致身故、伤残的，保险人不承担给付保险金责任：
（一）  战争（无论宣战与否）、军事行动、暴动或武装叛乱期间；
（二）  被保险人酒后驾驶、无合法有效驾驶证驾驶、或驾驶无合法有效行驶证的机动车期间；
（三）  被保险人醉酒或服用、吸食、注射毒品、管制药品的影响期间；被保险人精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）期间；
（四）  被保险人被国家机关依法拘禁、采取刑事强制措施或服刑期间；
（五）  被保险人无有效操作证操作施工设备期间；被保险人从事的工作或工种需当地监管部门颁发从业证件而无有效从业证件的工作期间；
（六）  被保险人作为职业运动员在参加训练或比赛期间；被保险人作为军人（含特种兵）、警务人员（含防暴警察）在训练或执行公务期间；被保险人从事采矿业、地下作业、山洞作业、水上作业、2米及以上高处作业的职业活动期间；
（七）  被保险人从事本合同内列明高危工种和职业所对应的工作或活动期间。
《  泰  康在线财产保险股份有限公司附加意外伤害救护车费用保险（互联网专属）条款  》  免责说明
下列费用或因下列原因造成被保险人支出救护车费用的，保险人不承担赔偿保险金责任：
（一）被保险人因疾病而发生的救护车费用；
（二）医生诊疗费、检查费、医药费、治疗费、担架费和转院时发生的费用（包括转院时发生的救护车车费）；
（三）一般身体检查、疗养、特别护理、静养、康复性治疗、物理治疗或心理治疗；
（四）主保险合同中规定的责任免除事项均适用于本附加合同。
《  泰康在线财产保险股份有限公司骨折意外伤害保险（互联网专属）条款  》  免责说明
因下列情形之一导致被保险人骨折的，保险人不承担给付保险金的责任：
（一）  投保人的故意行为；
（二）  因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；
（三）  被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；
（四）  被保险人未遵医嘱，私自服用、涂用、注射药物；
（五）  核爆炸、核辐射或核污染等放射性污染；
（六）  恐怖袭击；
（七）  被保险人犯罪或抗拒依法采取的刑事强制措施；
（八）  被保险人从事潜水、跳伞、滑翔、攀岩运动、武术比赛、探险活动、摔跤比赛、特技表演、赛马、马术表演、赛车、拳击等高风险运动或活动；
（九）  被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；
（十）  被保险人因精神错乱或失常而导致的意外；
（十一）  在中国境外的国家或者地区接受治疗；
（十二）  投保前已有骨折的治疗和康复；
（十三）  病理性骨折（指因疾病导致骨组织变弱的部位发生的任何骨折）。
《  泰康在线财产保险股份有限公司附加未成年人责任保险条款  》  免责说明
下列原因造成的损失、费用和责任，保险人不负责赔偿  ：
（一）  投保人、被保险人及监护人的故意行为；
（二）  与被保险人共同居住的成年家庭成员对被保险人的教唆或帮助被保险人损害第三者权益；
（三）  被保险人在精神错乱、神智不清、意识不清或智障状态下所造成的第三者人身伤亡、财产损失，不论该状态由何原因（包括但不限于疾病、服用药品或毒品、醉酒等）引起；
（四）  被保险人参加潜水、滑雪、滑板、滑翔、冲浪、蹦极、热气球、跳伞、攀岩、漂流、探险活动、特技表演、赛马、赛车等高风险运动，或进行摔跤、柔道、拳击、武术、散打、空手道、跆拳道等搏击运动，以及进行前述运动前准备活动时导致的损失、责任和费用；
（五）  战争、敌对行为、军事行为、武装冲突、罢工、骚乱、暴动、恐怖活动、盗窃、抢劫；
（六）  行政行为或司法行为；
（七）  核辐射、核爆炸、核污染及其他放射性污染；
（八）  地震、火山爆发、海啸、雷击、洪水、暴雨、台风、龙卷风、暴风、雪灾、雹灾、冰凌、泥石流、崩塌、突发性滑坡、地面突然下陷等自然灾害；
（九）  烟熏、大气污染、土地污染、水污染及其他各种污染；
（十）  被保险人自伤、自杀、打架、斗殴、犯罪。
（十一）  被保险人对其家庭成员、家庭雇佣人员、家庭暂居人员造成的人身伤亡或财产损失；
（十二）  被保险人所有、使用或管理的财物损失；
（十三）  被保险人或其家庭成员所有、使用或管理的机动车辆、飞机、船舶、武器或其宠物导致的人身伤亡或财产损失；
（十四）  被保险人使用计算机病毒、黑客等计算机技术所造成的损失、费用和责任；
（十五）  笔、打火机、手表、音像制品以及数码产品等财产损失，数码产品包括不限于智能手机、数码相机、平板电脑、摄像机、笔记本电脑、MP3、MP4、打印机、投影机等含有数码技术的电子产品；
（十六）  现金、金银、珠宝、玉器、钻石及制品、首饰；
（十七）  古币、古玩、字画、邮票、艺术品、稀有金属、各类收藏品等珍贵财物；
（十八）  票证、有价证券、文件、书籍、账册、图表、技术资料、电脑软件及资料、电子存储设备，以及无法鉴定价值的财产；
（十九）  被保险人的监护人应该承担的合同责任，但无合同存在时仍然应由被保险人的监护人承担的经济赔偿责任不在此限；
（二十）  罚款、罚金或惩罚性赔款；
（二十一）  精神损害赔偿；
（二十二）  第三者的间接损失；
（二十三）  被保险人在中华人民共和国境外造成的任何损失、责任和费用；
（二十四）  本附加合同中载明的免赔额或免赔率。
四  、投保声明
1、 本投保人 已完整阅读并了解以上投保须知 、产品说明 及投保险种的保险条款，尤其是对其中免除保险人责任的条款 或约定 （包括但不限于责任免除、投保人被保险人义务、保险金申请与给付等） ， 本 投保人 已充分理解并接受上述内容，同意以此作为订立保险合同的依据 ；
2、 投保时，本投保人已就该产品的保障内容以及保险金额等向被保险人/被保人监护人进行了明确说明，并征得其同意；
3、 投保单中所填写的内容均属实，如有隐瞒或不实告知，你公司有权解除保险合同，对于合同解除前发生的任何事故，你公司可不承担任何责任；
4、 本投保人已知晓本保险生效后退保有损失，投保人要求解除保险合同的，保险人计收保险责任开始之日起至合同解除之日止期间的保险费，并退还 未满期 保险费:
（1）保险责任开始前和在犹豫期内，投保人要求解除本合同的，保险人应当退还投保人已交纳的保险费；
（2）保险责任过犹豫期后，投保人要求解除保险合同的， 自保险人接到解除合同申请书之时起，保险合同解除， 保险人自收到解除保险合同申请之日起30日内向投保人退还本合同的未满期  保险费  ：
若保险费为一次性支付的：
未满期保险费=投保人已交纳保险费×[1-(保险单已生效天数/保险期间天数)]
若保险费为分期支付的：
未满期保险费=投保人已交纳当期保险费×[1-(保险单当期已经过天数/当期总天数)]
已生效  或已  经过天数不足一天的按一天计算。若本合同已发生保险金赔偿，未满期保险费为零
5、  本人同意贵公司为本保险及与本保险之相关服务的目的收集本人的个人资料，无论该资料是从本投保申请或其他地方所获取。本人理解贵公司为提升保险服务质量，可能会与第三方机构、组织或个人合作，此类合作可能需要使用本人的个人资料。在此理解下，本人同  意并授权贵公司及与贵公司存在合作关系的机构、组织或个人将收集的本人的个人资料用于：（1）该保险的投保审核；（2）该保险的理赔；（3）提供与该保险有关之服务；（4）与本人联络；
6、  本人同意泰康在线向中国保险信息技术管理有限责任公司（简称“中国保信”）报送本人的全部保单信息和理赔信息，并通过医疗机构、中国保信及知悉本人信息的其他机构查询与本人有关的承保、理赔、医疗等信息。中国保信基于为本人或保险公司提供服务的需要可对上述信息进行必要的使用及与相关机构进行信息共享，但均应严格履行保密义务  ；
7、 本人同意你公司通过手机（包括手机短信）、E-mail适时提供保险信息服务 ；
8、 电子发票送达方式：本产品提供电子发票，其法律效力、基本用途、基本使用规定等与税务机关监制的增值税普通发票相同，如您需要发票，您可以关注微信公众号[泰康在线保险]或者下载APP：
"""

deepai.chat(prompt)

KeyboardInterrupt: 

In [12]:
t1 = """
'以下是完整的免责声明部分，未做任何修改或遗漏：\n\n三、免责声明\n《泰康在线财产保险股份有限公司少儿医疗费用保险（互联网专属）》免责说明\n因下列情形之一导致被保险人支出医疗费用的，保险人不承担赔偿保险金的责任：\n（一）投保人对被保险人的故意杀害、故意伤害；\n（二）投保人或被保险人在投保时未如实告知的疾病；被保险人在投保前罹患且在投保时尚未治愈的疾病；\n（三）被保险人故意自伤，但被保险人自伤时为无民事行为能力人的除外；\n（四）被保险人故意犯罪或者抗拒依法采取的刑事强制措施，或被国家机关依法拘禁或服刑期间伤病；\n（五）被保险人感染艾滋病病毒或者患艾滋病、性病、精神和行为障碍、遗传性疾病、先天性畸形、变形或者染色体异常（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）为准）；\n（六）《中华人民共和国传染病防治法》规定的甲类及乙类法定传染病（不含病毒性肝炎），或者国家有关法律、法规、规范性法律文件规定的法定传染病。前述传染病定义以《中华人民共和国传染病防治法》或者国家有关法律、法规、规范性法律文件的规定为准；\n（七）被保险人接受疗养、康复治疗、心理治疗、美容、视力矫正手术、各种健康体检项目及预防性医疗项目、各种矫形及生理缺陷的手术和检查治疗项目、各种健美治疗项目、牙科保健及牙齿治疗以及非意外伤害事故所致的整容、整形手术；\n（八）被保险人接受如下项目的治疗：皮肤色素沉着；雀斑、痣的治疗和去除；对浅表静脉曲张、蜘蛛脉、除瘢痕疙瘩型外的其他瘢痕、纹身去除、皮肤变色的治疗或手术；包皮环切术、包皮剥离术、包皮气囊扩张术、性功能障碍治疗引起的医疗费；\n（九）被保险人患有性早熟、佝偻病、身材矮小、骨发育异常及其他发育迟缓疾患；\n（十）被保险人接受各类医疗鉴定，包括但不限于医疗事故鉴定、精神病鉴定、验伤鉴定、亲子鉴定、遗传基因鉴定所发生的费用；\n（十一）被保险人接受实验性治疗，即未经科学或医学认可的医疗；体外或植入的医疗辅助装置或用具（义肢、轮椅、拐杖、听器、眼镜或隐形眼镜、义眼等）及其安装费用；\n（十二）被保险人殴斗、醉酒，主动吸食或注射毒品，违反规定使用麻醉或精神药品；\n（十三）被保险人挑衅或者故意行为而导致的打斗、被袭击、被谋杀；\n（十四）被保险人酒后驾驶、无合法有效驾驶证驾驶，或者驾驶无合法有效行驶证的机动车导致交通意外引起的医疗费用；\n（十五）被保险人从事高风险运动导致的伤害引起的治疗；\n（十六）被保险人未遵医嘱私自服用、涂用、注射药物；\n（十七）被保险人由于职业病、医疗事故引起的医疗费用；\n（十八）战争、军事行动、暴乱、武装叛乱、核爆炸、核辐射、核污染；\n（十九）未经医生处方自行购药或非医院药房购药，医生开具的单次处方超过30天部分的药品费用。\n\n《泰康在线财产保险股份有限公司个人白血病疾病保险（互联网专属）》免责说明\n因下列情形之一导致被保险人罹患本合同定义的白血病的，保险人不承担给付保险金的责任：\n（一）投保人对被保险人的故意杀害、故意伤害；\n（二）被保险人故意自伤、故意犯罪或者抗拒依法采取的刑事强制措施或被国家机关依法拘禁或服刑期间伤病；\n（三）被保险人酗酒、殴斗、主动吸食或注射毒品，违反规定使用麻醉或精神药品；\n（四）战争（无论宣战与否）、军事冲突、暴乱或武装叛乱；\n（五）核爆炸、核辐射或核污染；\n（六）被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）、高血压III级期间；\n（七）被保险人患遗传性疾病，先天性畸形、变形或染色体异常（依据世界卫生组织《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）确定）；\n（八）未经科学或者医学认可的预防性治疗、实验性或试验性治疗及其产生的后果所导致的恶性肿瘤；\n（九）被保险人在投保时已经患有白血病的，但投保时保险人已知晓并做出书面认可的除外；\n（十）由于职业病、医疗事故引起。\n\n《泰康在线财产保险股份有限公司人身意外伤害保险E款（互联网专属）》免责说明：\n因下列原因造成被保险人身故、伤残的，保险人不承担给付保险金责任：\n（一）投保人的故意行为；\n（二）被保险人自伤或自杀，但被保险人自杀时为无民事行为能力人的除外；\n（三）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；\n（四）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；\n（五）被保险人未遵医嘱，私自服用、涂用、注射药物；\n（六）核爆炸、核辐射或核污染等放射性污染；\n（七）恐怖袭击；\n（八）被保险人犯罪或抗拒依法采取的刑事强制措施；\n（九）被保险人从事跳伞、滑翔、赛马、马术表演、赛车、拳击等高风险运动或活动；\n（十）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；\n（十一）被保险人因精神错乱或失常而导致的意外；\n（十二）被保险人非法搭乘交通工具或搭乘未经保险事故发生地相关政府部门登记许可的交通工具；\n（十三）被保险人因接受检查、麻醉、整容手术及其它内外科手术、药物治疗等导致的意外；\n（十四）被保险人因意外伤害事故以外的原因失踪而被法院宣告死亡的。\n被保险人在下列期间遭受意外伤害导致身故、伤残的，保险人不承担给付保险金责任：\n（一）战争（无论宣战与否）、军事行动、暴动或武装叛乱期间；\n（二）被保险人酒后驾驶、无合法有效驾驶证驾驶、或驾驶无合法有效行驶证的机动车期间；\n（三）被保险人醉酒或服用、吸食、注射毒品、管制药品的影响期间；被保险人精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）期间；\n（四）被保险人被国家机关依法拘禁、采取刑事强制措施或服刑期间；\n（五）被保险人无有效操作证操作施工设备期间；被保险人从事的工作或工种需当地监管部门颁发从业证件而无有效从业证件的工作期间；\n（六）被保险人作为职业运动员在参加训练或比赛期间；被保险人作为军人（含特种兵）、警务人员（含防暴警察）在训练或执行公务期间；被保险人从事采矿业、地下作业、山洞作业、水上作业、2米及以上高处作业的职业活动期间；\n（七）被保险人从事本合同内列明高危工种和职业所对应的工作或活动期间。\n\n《泰康在线财产保险股份有限公司附加意外伤害救护车费用保险（互联网专属）条款》免责说明\n下列费用或因下列原因造成被保险人支出救护车费用的，保险人不承担赔偿保险金责任：\n（一）被保险人因疾病而发生的救护车费用；\n（二）医生诊疗费、检查费、医药费、治疗费、担架费和转院时发生的费用（包括转院时发生的救护车车费）；\n（三）一般身体检查、疗养、特别护理、静养、康复性治疗、物理治疗或心理治疗；\n（四）主保险合同中规定的责任免除事项均适用于本附加合同。\n\n《泰康在线财产保险股份有限公司骨折意外伤害保险（互联网专属）条款》免责说明\n因下列情形之一导致被保险人骨折的，保险人不承担给付保险金的责任：\n（一）投保人的故意行为；\n（二）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；\n（三）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；\n（四）被保险人未遵医嘱，私自服用、涂用、注射药物；\n（五）核爆炸、核辐射或核污染等放射性污染；\n（六）恐怖袭击；\n（七）被保险人犯罪或抗拒依法采取的刑事强制措施；\n（八）被保险人从事潜水、跳伞、滑翔、攀岩运动、武术比赛、探险活动、摔跤比赛、特技表演、赛马、马术表演、赛车、拳击等高风险运动或活动；\n（九）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；\n（十）被保险人因精神错乱或失常而导致的意外；\n（十一）在中国境外的国家或者地区接受治疗；\n（十二）投保前已有骨折的治疗和康复；\n（十三）病理性骨折（指因疾病导致骨组织变弱的部位发生的任何骨折）。\n\n《泰康在线财产保险股份有限公司附加未成年人责任保险条款》免责说明\n下列原因造成的损失、费用和责任，保险人不负责赔偿：\n（一）投保人、被保险人及监护人的故意行为；\n（二）与被保险人共同居住的成年家庭成员对被保险人的教唆或帮助被保险人损害第三者权益；\n（三）被保险人在精神错乱、神智不清、意识不清或智障状态下所造成的第三者人身伤亡、财产损失，不论该状态由何原因（包括但不限于疾病、服用药品或毒品、醉酒等）引起；\n（四）被保险人参加潜水、滑雪、滑板、滑翔、冲浪、蹦极、热气球、跳伞、攀岩、漂流、探险活动、特技表演、赛马、赛车等高风险运动，或进行摔跤、柔道、拳击、武术、散打、空手道、跆拳道等搏击运动，以及进行前述运动前准备活动时导致的损失、责任和费用；\n（五）战争、敌对行为、军事行为、武装冲突、罢工、骚乱、暴动、恐怖活动、盗窃、抢劫；\n（六）行政行为或司法行为；\n（七）核辐射、核爆炸、核污染及其他放射性污染；\n（八）地震、火山爆发、海啸、雷击、洪水、暴雨、台风、龙卷风、暴风、雪灾、雹灾、冰凌、泥石流、崩塌、突发性滑坡、地面突然下陷等自然灾害；\n（九）烟熏、大气污染、土地污染、水污染及其他各种污染；\n（十）被保险人自伤、自杀、打架、斗殴、犯罪。\n（十一）被保险人对其家庭成员、家庭雇佣人员、家庭暂居人员造成的人身伤亡或财产损失；\n（十二）被保险人所有、使用或管理的财物损失；\n（十三）被保险人或其家庭成员所有、使用或管理的机动车辆、飞机、船舶、武器或其宠物导致的人身伤亡或财产损失；\n（十四）被保险人使用计算机病毒、黑客等计算机技术所造成的损失、费用和责任；\n（十五）笔、打火机、手表、音像制品以及数码产品等财产损失，数码产品包括不限于智能手机、数码相机、平板电脑、摄像机、笔记本电脑、MP3、MP4、打印机、投影机等含有数码技术的电子产品；\n（十六）现金、金银、珠宝、玉器、钻石及制品、首饰；\n（十七）古币、古玩、字画、邮票、艺术品、稀有金属、各类收藏品等珍贵财物；\n（十八）票证、有价证券、文件、书籍、账册、图表、技术资料、电脑软件及资料、电子存储设备，以及无法鉴定价值的财产；\n（十九）被保险人的监护人应该承担的合同责任，但无合同存在时仍然应由被保险人的监护人承担的经济赔偿责任不在此限；\n（二十）罚款、罚金或惩罚性赔款；\n（二十一）精神损害赔偿；\n（二十二）第三者的间接损失；\n（二十三）被保险人在中华人民共和国境外造成的任何损失、责任和费用；\n（二十四）本附加合同中载明的免赔额或免赔率。'
"""
t2 = """
责任免除
	层级1	层级2	层级3	层级4
	"门（急）诊医疗费用保险金、住院医疗保险金"保障的免责条款以《泰康在线财产保险股份有限公司少儿医疗费用保险（互联网专属）条款》为准，具体如下：
		因下列情形之一导致被保险人支出医疗费用的，保险人不承担赔偿保险金的责任：
			（一）投保人对被保险人的故意杀害、故意伤害；
			（二）投保人或被保险人在投保时未如实告知的疾病；被保险人在投保前罹患且在投保时尚未治愈的疾病；
			（三）被保险人故意自伤，但被保险人自伤时为无民事行为能力人的除外；
			（四）被保险人故意犯罪或者抗拒依法采取的刑事强制措施，或被国家机关依法拘禁或服刑期间伤病；
			（五）被保险人感染艾滋病病毒或者患艾滋病、性病、精神和行为障碍、遗传性疾病、先天性畸形、变形或者染色体异常（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）为准）；
			（六）《中华人民共和国传染病防治法》规定的甲类及乙类法定传染病（不含病毒性肝炎），或者国家有关法律、法规、规范性法律文件规定的法定传染病。前述传染病定义以《中华人民共和国传染病防治法》或者国家有关法律、法规、规范性法律文件的规定为准；
			（七）被保险人接受疗养、康复治疗、心理治疗、美容、视力矫正手术、各种健康体检项目及预防性医疗项目、各种矫形及生理缺陷的手术和检查治疗项目、各种健美治疗项目、牙科保健及牙齿治疗以及非意外伤害事故所致的整容、整形手术；
			（八）被保险人接受如下项目的治疗：皮肤色素沉着；雀斑、痣的治疗和去除；对浅表静脉曲张、蜘蛛脉、除瘢痕疙瘩型外的其他瘢痕、纹身去除、皮肤变色的治疗或手术；包皮环切术、包皮剥离术、包皮气囊扩张术、性功能障碍治疗引起的医疗费；
			（九）被保险人患有性早熟、佝偻病、身材矮小、骨发育异常及其他发育迟缓疾患；
			（十）被保险人接受各类医疗鉴定，包括但不限于医疗事故鉴定、精神病鉴定、验伤鉴定、亲子鉴定、遗传基因鉴定所发生的费用；
			（十一）被保险人接受实验性治疗，即未经科学或医学认可的医疗；体外或植入的医疗辅助装置或用具（义肢、轮椅、拐杖、听器、眼镜或隐形眼镜、义眼等）及其安装费用：
			（十二）被保险人殴斗、醉酒，主动吸食或注射毒品，违反规定使用麻醉或精神药品；
			（十三）被保险人挑衅或者故意行为而导致的打斗、被袭击、被谋杀；
			（十四）被保险人酒后驾驶、无合法有效驾驶证驾驶，或者驾驶无合法有效行驶证的机动车导致交通意外引起的医疗费用；
			（十五）被保险人从事高风险运动导致的伤害引起的治疗；
			（十六）被保险人未遵医嘱私自服用、涂用、注射药物；
			（十七）被保险人由于职业病、医疗事故引起的医疗费用；
			（十八）战争、军事行动、暴乱、武装叛乱、核爆炸、核辐射、核污染；
			（十九）未经医生处方自行购药或非医院药房购药，医生开具的单次处方超过30天部分的药品费用。
	"白血病疾病保险金"保障的免责条款以《泰康在线财产保险股份有限公司白血病疾病保险条款》为准，具体如下：
		因下列情形之一导致被保险人罹患本合同定义的白血病的，保险人不承担给付保险金的责任：
			（一）投保人对被保险人的故意杀害、故意伤害；
			（二）被保险人故意自伤、故意犯罪或者抗拒依法采取的刑事强制措施或被国家机关依法拘禁或服刑期间伤病；
			（三）被保险人酗酒、殴斗、主动吸食或注射毒品，违反规定使用麻醉或精神药品；
			（四）战争（无论宣战与否）、军事冲突、暴乱或武装叛乱；
			（五）核爆炸、核辐射或核污染；
			（六）被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）期间；
			（七）被保险人患遗传性疾病，先天性畸形、变形或染色体异常（依据世界卫生组织《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）确定）；
			（八）未经科学或者医学认可的预防性治疗、实验性或试验性治疗及其产生的后果所导致的恶性肿瘤；
			（九）被保险人在投保时已经患有白血病的，但投保时保险人已知晓并做出书面认可的除外；
			（十）由于职业病、医疗事故引起。
	"意外身故保险金、意外伤残保险金"保障的免责条款以《泰康在线财产保险股份有限公司人身意外伤害保险E款（互联网专属）条款》为准，具体如下：
		因下列原因造成被保险人身故、伤残的，保险人不承担给付保险金责任：
			（一）投保人的故意行为；
			（二）被保险人自伤或自杀，但被保险人自杀时为无民事行为能力人的除外；
			（三）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；
			（四）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；
			（五）被保险人未遵医嘱，私自服用、涂用、注射药物；
			（六）核爆炸、核辐射或核污染等放射性污染；
			（七）恐怖袭击；
			（八）被保险人犯罪或抗拒依法采取的刑事强制措施；
			（九）被保险人从事跳伞、滑翔、赛马、马术表演、赛车、拳击等高风险运动或活动；
			（十）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；
			（十一）被保险人因精神错乱或失常而导致的意外；
			（十二）被保险人非法搭乘交通工具或搭乘未经保险事故发生地相关政府部门登记许可的交通工具；
			（十三）被保险人因接受检查、麻醉、整容手术及其它内外科手术、药物治疗等导致的意外；
			（十四）被保险人因意外伤害事故以外的原因失踪而被法院宣告死亡的。
	"附加意外伤害救护车费用保险金"保障的免责条款以《泰康在线财产保险股份有限公司附加意外伤害救护车费用保险（互联网专属）条款》为准，具体如下：
		下列费用或因下列原因造成被保险人支出救护车费用的，保险人不承担赔偿保险金责任：
			（一）被保险人因疾病而发生的救护车费用；
			（二）医生诊疗费、检查费、医药费、治疗费、担架费和转院时发生的费用（包括转院时发生的救护车车费）；
			（三）一般身体检查、疗养、特别护理、静养、康复性治疗、物理治疗或心理治疗；
			（四）主保险合同中规定的责任免除事项均适用于本附加合同。
	"骨折意外伤害保险金"保障的免责条款以《泰康在线财产保险股份有限公司骨折意外伤害保险（互联网专属）条款》为准，具体如下：
		因下列情形之一导致被保险人骨折的，保险人不承担给付保险金的责任：
			（一）投保人的故意行为；
			（二）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；
			（三）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；
			（四）被保险人未遵医嘱，私自服用、涂用、注射药物；
			（五）核爆炸、核辐射或核污染等放射性污染；
			（六）恐怖袭击；
			（七）被保险人犯罪或抗拒依法采取的刑事强制措施；
			（八）被保险人从事潜水、跳伞、滑翔、攀岩运动、武术比赛、探险活动、摔跤比赛、特技表演、赛马、马术表演、赛车、拳击等高风险运动或活动；
			（九）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；
			（十）被保险人因精神错乱或失常而导致的意外；
			（十一）在中国境外的国家或者地区接受治疗；
			（十二）投保前已有骨折的治疗和康复；
			（十三）病理性骨折（指因疾病导致骨组织变弱的部位发生的任何骨折）。
	"附加未成年人责任保险金"保障的免责条款以《泰康在线财产保险股份有限公司附加未成年人责任条款》为准，具体如下：
		下列原因造成的损失、费用和责任，保险人不负责赔偿：
			（一）投保人、被保险人及监护人的故意行为；
			（二）与被保险人共同居住的成年家庭成员对被保险人的教唆或帮助被保险人损害第三者权益；
			（三）被保险人在精神错乱、神智不清、意识不清或智障状态下所造成的第三者人身伤亡、财产损失，不论该状态由何原因（包括但不限于疾病、服用药品或毒品、醉酒等）引起；
			（四）被保险人参加潜水、滑雪、滑板、滑翔、冲浪、蹦极、热气球、跳伞、攀岩、漂流、探险活动、特技表演、赛马、赛车等高风险运动，或进行摔跤、柔道、拳击、武术、散打、空手道、跆拳道等搏击运动，以及进行前述运动前准备活动时导致的损失、责任和费用；
			（五）战争、敌对行为、军事行为、武装冲突、罢工、骚乱、暴动、恐怖活动、盗窃、抢劫；
			（六）行政行为或司法行为；
			（七）核辐射、核爆炸、核污染及其他放射性污染；
			（八）地震、火山爆发、海啸、雷击、洪水、暴雨、台风、龙卷风、暴风、雪灾、雹灾、冰凌、泥石流、崩塌、突发性滑坡、地面突然下陷等自然灾害；
			（九）烟熏、大气污染、土地污染、水污染及其他各种污染；
			（十）被保险人自伤、自杀、打架、斗殴、犯罪。

填写说明
	填写说明
	1、请将需要Excel填写的内容，填写在第一个表单，例如免责说明
	2、表单分为几个层级，层级1是大标题，层级2是文案，层级3是子文案
	3、请按顺序填写内容，页面将按照您填写的顺序渲染
	4、每行仅填写一个单元格，参考第一页的样例
"""

In [15]:
prompt = f"""详细对比下面两份免责条款（约定该产品不承担保险责任的情形）文件中，查找是否有不一致之处，即：同一个概念在不同文件中的定义语义不一致，有明显的矛盾之处,有的话请指出来：
文件1：
{t1}
=================================================================================================
文件2：
{t2}
"""

In [16]:
deepai.chat(prompt)

'经过详细对比，发现两份文件中关于 **"白血病疾病保险金"的免责条款** 存在一处明显的不一致：\n\n### 不一致点：\n**文件1** 中《泰康在线财产保险股份有限公司个人白血病疾病保险（互联网专属）》免责说明的 **第（六）条** 包含：\n> （六）被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）、高血压III级期间；\n\n**文件2** 中对应条款的 **第（六）条** 为：\n> （六）被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）期间；\n\n### 差异分析：\n1. **高血压III级期间** 仅在文件1中被列为免责情形，文件2中未提及。\n2. 文件2的条款名称与文件1不完全一致（文件2为《白血病疾病保险条款》，文件1为《个人白血病疾病保险（互联网专属）》），但核心免责内容应保持一致。\n3. 其他条款（如遗传性疾病、战争等）在两份文件中完全一致，仅此条存在额外限制。\n\n### 影响：\n若以文件2为准，高血压III级患者可能未被明确排除在白血病保险责任外，可能导致理赔争议。需确认是否为条款修订或录入遗漏。\n\n### 其他说明：\n其余免责条款（如医疗费用、意外伤害等）在两份文件中内容完全一致，未发现其他矛盾。'

In [ ]:
"""
'以下是完整的免责声明部分，未做任何修改或遗漏：\n\n三、免责声明\n《泰康在线财产保险股份有限公司少儿医疗费用保险（互联网专属）》免责说明\n因下列情形之一导致被保险人支出医疗费用的，保险人不承担赔偿保险金的责任：\n（一）投保人对被保险人的故意杀害、故意伤害；\n（二）投保人或被保险人在投保时未如实告知的疾病；被保险人在投保前罹患且在投保时尚未治愈的疾病；\n（三）被保险人故意自伤，但被保险人自伤时为无民事行为能力人的除外；\n（四）被保险人故意犯罪或者抗拒依法采取的刑事强制措施，或被国家机关依法拘禁或服刑期间伤病；\n（五）被保险人感染艾滋病病毒或者患艾滋病、性病、精神和行为障碍、遗传性疾病、先天性畸形、变形或者染色体异常（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）为准）；\n（六）《中华人民共和国传染病防治法》规定的甲类及乙类法定传染病（不含病毒性肝炎），或者国家有关法律、法规、规范性法律文件规定的法定传染病。前述传染病定义以《中华人民共和国传染病防治法》或者国家有关法律、法规、规范性法律文件的规定为准；\n（七）被保险人接受疗养、康复治疗、心理治疗、美容、视力矫正手术、各种健康体检项目及预防性医疗项目、各种矫形及生理缺陷的手术和检查治疗项目、各种健美治疗项目、牙科保健及牙齿治疗以及非意外伤害事故所致的整容、整形手术；\n（八）被保险人接受如下项目的治疗：皮肤色素沉着；雀斑、痣的治疗和去除；对浅表静脉曲张、蜘蛛脉、除瘢痕疙瘩型外的其他瘢痕、纹身去除、皮肤变色的治疗或手术；包皮环切术、包皮剥离术、包皮气囊扩张术、性功能障碍治疗引起的医疗费；\n（九）被保险人患有性早熟、佝偻病、身材矮小、骨发育异常及其他发育迟缓疾患；\n（十）被保险人接受各类医疗鉴定，包括但不限于医疗事故鉴定、精神病鉴定、验伤鉴定、亲子鉴定、遗传基因鉴定所发生的费用；\n（十一）被保险人接受实验性治疗，即未经科学或医学认可的医疗；体外或植入的医疗辅助装置或用具（义肢、轮椅、拐杖、听器、眼镜或隐形眼镜、义眼等）及其安装费用；\n（十二）被保险人殴斗、醉酒，主动吸食或注射毒品，违反规定使用麻醉或精神药品；\n（十三）被保险人挑衅或者故意行为而导致的打斗、被袭击、被谋杀；\n（十四）被保险人酒后驾驶、无合法有效驾驶证驾驶，或者驾驶无合法有效行驶证的机动车导致交通意外引起的医疗费用；\n（十五）被保险人从事高风险运动导致的伤害引起的治疗；\n（十六）被保险人未遵医嘱私自服用、涂用、注射药物；\n（十七）被保险人由于职业病、医疗事故引起的医疗费用；\n（十八）战争、军事行动、暴乱、武装叛乱、核爆炸、核辐射、核污染；\n（十九）未经医生处方自行购药或非医院药房购药，医生开具的单次处方超过30天部分的药品费用。\n\n《泰康在线财产保险股份有限公司个人白血病疾病保险（互联网专属）》免责说明\n因下列情形之一导致被保险人罹患本合同定义的白血病的，保险人不承担给付保险金的责任：\n（一）投保人对被保险人的故意杀害、故意伤害；\n（二）被保险人故意自伤、故意犯罪或者抗拒依法采取的刑事强制措施或被国家机关依法拘禁或服刑期间伤病；\n（三）被保险人酗酒、殴斗、主动吸食或注射毒品，违反规定使用麻醉或精神药品；\n（四）战争（无论宣战与否）、军事冲突、暴乱或武装叛乱；\n（五）核爆炸、核辐射或核污染；\n（六）被保险人感染艾滋病病毒（HIV）或患艾滋病（AIDS）、高血压III级期间；\n（七）被保险人患遗传性疾病，先天性畸形、变形或染色体异常（依据世界卫生组织《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）确定）；\n（八）未经科学或者医学认可的预防性治疗、实验性或试验性治疗及其产生的后果所导致的恶性肿瘤；\n（九）被保险人在投保时已经患有白血病的，但投保时保险人已知晓并做出书面认可的除外；\n（十）由于职业病、医疗事故引起。\n\n《泰康在线财产保险股份有限公司人身意外伤害保险E款（互联网专属）》免责说明：\n因下列原因造成被保险人身故、伤残的，保险人不承担给付保险金责任：\n（一）投保人的故意行为；\n（二）被保险人自伤或自杀，但被保险人自杀时为无民事行为能力人的除外；\n（三）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；\n（四）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；\n（五）被保险人未遵医嘱，私自服用、涂用、注射药物；\n（六）核爆炸、核辐射或核污染等放射性污染；\n（七）恐怖袭击；\n（八）被保险人犯罪或抗拒依法采取的刑事强制措施；\n（九）被保险人从事跳伞、滑翔、赛马、马术表演、赛车、拳击等高风险运动或活动；\n（十）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；\n（十一）被保险人因精神错乱或失常而导致的意外；\n（十二）被保险人非法搭乘交通工具或搭乘未经保险事故发生地相关政府部门登记许可的交通工具；\n（十三）被保险人因接受检查、麻醉、整容手术及其它内外科手术、药物治疗等导致的意外；\n（十四）被保险人因意外伤害事故以外的原因失踪而被法院宣告死亡的。\n被保险人在下列期间遭受意外伤害导致身故、伤残的，保险人不承担给付保险金责任：\n（一）战争（无论宣战与否）、军事行动、暴动或武装叛乱期间；\n（二）被保险人酒后驾驶、无合法有效驾驶证驾驶、或驾驶无合法有效行驶证的机动车期间；\n（三）被保险人醉酒或服用、吸食、注射毒品、管制药品的影响期间；被保险人精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）期间；\n（四）被保险人被国家机关依法拘禁、采取刑事强制措施或服刑期间；\n（五）被保险人无有效操作证操作施工设备期间；被保险人从事的工作或工种需当地监管部门颁发从业证件而无有效从业证件的工作期间；\n（六）被保险人作为职业运动员在参加训练或比赛期间；被保险人作为军人（含特种兵）、警务人员（含防暴警察）在训练或执行公务期间；被保险人从事采矿业、地下作业、山洞作业、水上作业、2米及以上高处作业的职业活动期间；\n（七）被保险人从事本合同内列明高危工种和职业所对应的工作或活动期间。\n\n《泰康在线财产保险股份有限公司附加意外伤害救护车费用保险（互联网专属）条款》免责说明\n下列费用或因下列原因造成被保险人支出救护车费用的，保险人不承担赔偿保险金责任：\n（一）被保险人因疾病而发生的救护车费用；\n（二）医生诊疗费、检查费、医药费、治疗费、担架费和转院时发生的费用（包括转院时发生的救护车车费）；\n（三）一般身体检查、疗养、特别护理、静养、康复性治疗、物理治疗或心理治疗；\n（四）主保险合同中规定的责任免除事项均适用于本附加合同。\n\n《泰康在线财产保险股份有限公司骨折意外伤害保险（互联网专属）条款》免责说明\n因下列情形之一导致被保险人骨折的，保险人不承担给付保险金的责任：\n（一）投保人的故意行为；\n（二）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；\n（三）被保险人妊娠、流产、分娩、疾病、药物过敏、中暑、食物中毒、猝死；\n（四）被保险人未遵医嘱，私自服用、涂用、注射药物；\n（五）核爆炸、核辐射或核污染等放射性污染；\n（六）恐怖袭击；\n（七）被保险人犯罪或抗拒依法采取的刑事强制措施；\n（八）被保险人从事潜水、跳伞、滑翔、攀岩运动、武术比赛、探险活动、摔跤比赛、特技表演、赛马、马术表演、赛车、拳击等高风险运动或活动；\n（九）被保险人参加任何职业或半职业体育运动或设有奖金、报酬的体育活动；\n（十）被保险人因精神错乱或失常而导致的意外；\n（十一）在中国境外的国家或者地区接受治疗；\n（十二）投保前已有骨折的治疗和康复；\n（十三）病理性骨折（指因疾病导致骨组织变弱的部位发生的任何骨折）。\n\n《泰康在线财产保险股份有限公司附加未成年人责任保险条款》免责说明\n下列原因造成的损失、费用和责任，保险人不负责赔偿：\n（一）投保人、被保险人及监护人的故意行为；\n（二）与被保险人共同居住的成年家庭成员对被保险人的教唆或帮助被保险人损害第三者权益；\n（三）被保险人在精神错乱、神智不清、意识不清或智障状态下所造成的第三者人身伤亡、财产损失，不论该状态由何原因（包括但不限于疾病、服用药品或毒品、醉酒等）引起；\n（四）被保险人参加潜水、滑雪、滑板、滑翔、冲浪、蹦极、热气球、跳伞、攀岩、漂流、探险活动、特技表演、赛马、赛车等高风险运动，或进行摔跤、柔道、拳击、武术、散打、空手道、跆拳道等搏击运动，以及进行前述运动前准备活动时导致的损失、责任和费用；\n（五）战争、敌对行为、军事行为、武装冲突、罢工、骚乱、暴动、恐怖活动、盗窃、抢劫；\n（六）行政行为或司法行为；\n（七）核辐射、核爆炸、核污染及其他放射性污染；\n（八）地震、火山爆发、海啸、雷击、洪水、暴雨、台风、龙卷风、暴风、雪灾、雹灾、冰凌、泥石流、崩塌、突发性滑坡、地面突然下陷等自然灾害；\n（九）烟熏、大气污染、土地污染、水污染及其他各种污染；\n（十）被保险人自伤、自杀、打架、斗殴、犯罪。\n（十一）被保险人对其家庭成员、家庭雇佣人员、家庭暂居人员造成的人身伤亡或财产损失；\n（十二）被保险人所有、使用或管理的财物损失；\n（十三）被保险人或其家庭成员所有、使用或管理的机动车辆、飞机、船舶、武器或其宠物导致的人身伤亡或财产损失；\n（十四）被保险人使用计算机病毒、黑客等计算机技术所造成的损失、费用和责任；\n（十五）笔、打火机、手表、音像制品以及数码产品等财产损失，数码产品包括不限于智能手机、数码相机、平板电脑、摄像机、笔记本电脑、MP3、MP4、打印机、投影机等含有数码技术的电子产品；\n（十六）现金、金银、珠宝、玉器、钻石及制品、首饰；\n（十七）古币、古玩、字画、邮票、艺术品、稀有金属、各类收藏品等珍贵财物；\n（十八）票证、有价证券、文件、书籍、账册、图表、技术资料、电脑软件及资料、电子存储设备，以及无法鉴定价值的财产；\n（十九）被保险人的监护人应该承担的合同责任，但无合同存在时仍然应由被保险人的监护人承担的经济赔偿责任不在此限；\n（二十）罚款、罚金或惩罚性赔款；\n（二十一）精神损害赔偿；\n（二十二）第三者的间接损失；\n（二十三）被保险人在中华人民共和国境外造成的任何损失、责任和费用；\n（二十四）本附加合同中载明的免赔额或免赔率。'
"""